In [22]:
import pandas as pd
import numpy as np

In [122]:
films0 = pd.read_csv('myshows_checkpoints/checkpoint_1200.csv')
films1 = pd.read_csv('checkpoints/checkpoint_41100.csv')
films2 = pd.read_csv('checkpoints_41k_plus/checkpoint_875000.csv')

In [123]:
films = pd.concat([films0, films1, films2], ignore_index=True)

In [131]:
films.isna().sum()

url                         0
title                    4203
title_en               822373
Дата выхода:             4244
Страна:                334007
Жанры:                 236288
Производство:          451268
Смотрящих:               4244
Длительность:          197583
Рейтинг IMDB:          487236
myshows_rating         884890
myshows_top            884890
poster_url             166247
reaction_unamused           0
reaction_relaxed            0
reaction_thinking           0
reaction_grimacing          0
reaction_loving             0
reaction_anxious            0
reaction_hugging            0
reaction_starstruck         0
reaction_laughing           0
reaction_flushed            0
reaction_grateful           0
reaction_crying             0
reaction_mindblown          0
reaction_silent             0
reaction_disgusted          0
actors                 275622
description            811894
Рейтинг Кинопоиска:    876729
dtype: int64

In [132]:
films.title.count()

889343

In [133]:
films = films.drop_duplicates(subset=['title'], keep='first', inplace=False, ignore_index=False)

In [137]:
len(films.title.unique()), films.title.count()

(787830, 787829)

In [138]:
films['Дата выхода:'].fillna('', inplace=True)

/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/1542343540.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  films['Дата выхода:'].fillna('', inplace=True)
/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/1542343540.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films['Дата выхода:'].fillna('', inplace=T

In [139]:
films['Дата выхода:'].isna().sum()

0

In [141]:
months_en = {'января': 'january', 
             'февраля': 'february',
             'марта': 'march',
             'апреля': 'april',
             'мая': 'may',
             'июня': 'june',
             'июля': 'july',
             'августа': 'august',
             'сентября': 'september',
             'октября': 'october',
             'ноября': 'november',
             'декабря': 'december'}

films['Дата выхода:'] = films['Дата выхода:'].apply(
    lambda x: (
        x.replace(x.split(' ')[1], months_en.get(x.split(' ')[1], x.split(' ')[1]))
        if isinstance(x, str) and len(x.split(' ')) > 1
        else x
    )
)

/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/1183799633.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films['Дата выхода:'] = films['Дата выхода:'].apply(


In [142]:
films['Дата выхода:'] = pd.to_datetime(films['Дата выхода:'], dayfirst=True, errors='coerce')

/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/3625629836.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  films['Дата выхода:'] = pd.to_datetime(films['Дата выхода:'], dayfirst=True, errors='coerce')
/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/3625629836.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films['Дата выхода:'] = pd.to_datetime(films['Дата выхода:'], dayfirst=True, errors='coerce')


In [143]:
films['Дата выхода:']

0        2022-02-10
1        2022-04-08
2        2021-10-13
3        2019-07-12
4        2021-11-24
            ...    
893541   2002-07-22
893542   2020-12-14
893543   2025-04-13
893544   2020-06-09
893545          NaT
Name: Дата выхода:, Length: 787830, dtype: datetime64[ns]

In [145]:
films['Жанры:'] = films['Жанры:'].apply(lambda x: x.split(',') if isinstance(x, str) else x)

/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/3297677547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films['Жанры:'] = films['Жанры:'].apply(lambda x: x.split(',') if isinstance(x, str) else x)


In [146]:
def convert_time(x):
    if pd.isna(x):
        return np.nan

    digits = [int(el) for el in x.split() if el.isdigit()]
    
    if len(digits) == 2:
        return digits[0] * 60 + digits[1]  # e.g. "1 ч 30 мин"
    elif len(digits) == 1:
        return digits[0]  # assume it's in minutes
    else:
        return np.nan  # if nothing was found


films['Длительность:'] = films['Длительность:'].apply(lambda x: convert_time(x))

/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/3842510057.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films['Длительность:'] = films['Длительность:'].apply(lambda x: convert_time(x))


In [147]:
def parse_ratings(x):
    if pd.isna(x):
        return pd.Series([0, 0])
    parts = x.strip().split('из 10')
    rating = parts[0].strip()
    score = parts[1].strip() if len(parts) > 1 else 0
    return pd.Series([rating, score])

In [148]:
films[['imdb_rating', 'imdb_scores']] = films['Рейтинг IMDB:'].apply(parse_ratings)
films[['kp_rating', 'kp_scores']] = films['Рейтинг Кинопоиска:'].apply(parse_ratings)

/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/1057128691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films[['imdb_rating', 'imdb_scores']] = films['Рейтинг IMDB:'].apply(parse_ratings)
/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/1057128691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films[['imdb_rating', 'imdb_scores']] = films['Рейтинг IMDB:'].apply(parse_ratings)
/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/1057128691.py:2: Settin

In [149]:
films['actors'] = films['actors'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)
films['Страна:'] = films['Страна:'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
films['Производство:'] = films['Производство:'].apply(lambda x: x.split(',') if isinstance(x, str) else x)


/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/4250308618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films['actors'] = films['actors'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)
/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/4250308618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films['Страна:'] = films['Страна:'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
/var/folders/wm/2tqt73c11jngw1rs6w9r_v0w0000gn/T/ipykernel_23572/42

In [150]:
films

,url,title,title_en,Дата выхода:,Страна:,Жанры:,Производство:,Смотрящих:,Длительность:,Рейтинг IMDB:,...,reaction_mindblown,reaction_silent,reaction_disgusted,actors,description,Рейтинг Кинопоиска:,imdb_rating,imdb_scores,kp_rating,kp_scores
0,https://myshows.me/movie/58,Кими,Kimi,2022-02-10,[США],"[Триллер, Детектив, Криминал]","[New Line Cinema, HBO Max]",334 из 923 026,89.0,6.3из 1041 153,...,0,0,0,"[Зои Кравиц, Байрон Бауэрс, Jaime Camil, Джейк...","В центре сюжета фильма — Анджела Чайлдс, котор...",NaN,6.3,41 153,0,0
1,https://myshows.me/movie/128,Хрустальные девушки,Las niñas de cristal,2022-04-08,[Испания],[Драма],[Federation Spain],47 из 923 026,137.0,5.6из 101 093,...,0,0,0,"[María Pedraza, Paula Losada, Mona Martínez, М...","Балерина получает главную партию и, в попытке ...",NaN,5.6,1 093,0,0
2,https://myshows.me/movie/21,Энканто,Encanto,2021-10-13,[США],"[Мультфильмы, Комедия, Семейный, Фэнтези]","[Walt Disney Animation Studios, Walt Disney Pi...",9 997 из 923 026,99.0,7.2из 10280 529,...,0,0,0,"[Стефани Беатриз, María Cecilia Botero, Wilmer...",Удивительная семья Мадригалов живет в спрятанн...,7.49из 10160 133,7.2,280 529,7.49,160 133
3,https://myshows.me/movie/272,Покемон: Мьюту наносит ответный удар – Эволюция,ミュウツーの逆襲 EVOLUTION,2019-07-12,[Япония],"[Мультфильмы, Приключения, Фэнтези, Боевик, Се...","[OLM, The Pokémon Company, Shogakukan, TV Toky...",61 из 923 026,98.0,5.7из 106 627,...,0,0,0,"[松本梨香, 大谷育江, 飯塚雅弓, こおろぎさとみ, Юдзи Уэда]",Гениальный учёный работает над созданием живог...,NaN,5.7,6 627,0,0
4,https://myshows.me/movie/56,Обитель зла: Раккун-Сити,Resident Evil: Welcome to Raccoon City,2021-11-24,"[Франция, Германия, США]","[Боевик, Ужасы, Фантастика]","[Constantin Film, Davis Films, Screen Gems]",1 743 из 923 026,107.0,5.2из 1068 171,...,0,2,4,"[Кая Скоделарио, Робби Амелл, Том Хоппер, Эван...",30 сентября 1998 года. Клэр Редфилд возвращает...,5.4из 1048 836,5.2,68 171,5.4,48 836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893541,https://myshows.me/movie/913438,High Performance Imports: Volume 4,NaN,2002-07-22,NaN,[Документальный],[Otomoto],0 из 923 189,54.0,NaN,...,0,0,0,NaN,NaN,NaN,0,0,0,0
893542,https://myshows.me/movie/911429,颜如玉,NaN,2020-12-14,[Китай],NaN,"[Xinmei Media, Yuxun Films, Tibet Ruiyi Culture]",0 из 923 189,82.0,NaN,...,0,0,0,"[Su Delun, Chen Kairui, Wu Xiaoyuan, Mu Yuansong]",NaN,NaN,0,0,0,0
893543,https://myshows.me/movie/912674,GOTHURUTH,NaN,2025-04-13,NaN,NaN,[4C's STUDIOS],0 из 923 189,32.0,NaN,...,0,0,0,NaN,NaN,NaN,0,0,0,0
893544,https://myshows.me/movie/912484,Room 831,NaN,2020-06-09,NaN,NaN,NaN,0 из 923 189,4.0,NaN,...,0,0,0,[Martin Angerbauer],NaN,NaN,0,0,0,0


In [151]:
films_final = films[films['description'].isna() == False]

In [152]:
films_final.to_csv('films_all.csv', index=False)